## Huffman Constructor

Huffman encoding can optimize the TapTree structure, and thus potentially lower the size of revealed branches, by taking into account the estimated frequency by which each TapLeaf will occur. Since some spending scenarios are more likely to occur than others, ideally we would optimize the tree structure such that those spends appear closer to the root of the tree, and therefore require a smaller merkle proof when spending.

We reconstruct the same taptree from the above examples using the `huffman_constructor()`. That function takes a list of `(weight,tapleaf)` tuples, where `weight` is an `int` and `tapleaf` is a `TapLeaf` object. A higher weight:
   
* Indicates a higher likelihood of execution
* Means the script will be placed closer to the root if possible
* Results in a smaller inclusion proof and lower spending fees

![test](images/TapTree3.jpg)